<a href="https://colab.research.google.com/github/carlosperz88/dasany/blob/main/Sincronizador_de_Im%C3%A1genes_y_M%C3%BAsica_(Estilo_TikTok_B%C3%A1sico).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Sincronizador de Imágenes y Música (Estilo TikTok Básico)

# -*- coding: utf-8 -*-
"""
Cuaderno para crear un video slideshow sincronizado con el ritmo de la música.
Usa librosa para detectar beats y moviepy para crear el video.
Interfaz creada con Gradio.
"""

# 1. Instalaciones
print("Instalando librerías: gradio, moviepy, librosa, Pillow...")
!pip install gradio moviepy librosa Pillow -q
print("Instalación completada.")

# 2. Importaciones
import gradio as gr
from moviepy.editor import *
import librosa
import numpy as np
from PIL import Image
import os
import time
import shutil

print("Librerías importadas.")

# Directorio para salidas temporales
output_dir = "synced_video_outputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def clear_output_dir():
    """Limpia el directorio de salida."""
    if os.path.exists(output_dir):
        for filename in os.listdir(output_dir):
            file_path = os.path.join(output_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Error al borrar {file_path}. Razón: {e}')

def resize_image(image_path, target_size=(1280, 720)):
    """Redimensiona una imagen manteniendo la relación de aspecto y añadiendo barras negras si es necesario."""
    try:
        img = Image.open(image_path).convert('RGB')
        img.thumbnail(target_size, Image.Resampling.LANCZOS)

        new_img = Image.new('RGB', target_size, (0, 0, 0))
        paste_x = (target_size[0] - img.width) // 2
        paste_y = (target_size[1] - img.height) // 2
        new_img.paste(img, (paste_x, paste_y))

        temp_resized_path = f"{os.path.splitext(image_path)[0]}_resized.png"
        new_img.save(temp_resized_path, "PNG")
        return temp_resized_path
    except Exception as e:
        print(f"Error redimensionando {image_path}: {e}")
        return None

def create_synced_video(audio_file, image_files, max_duration_seconds, images_per_beat, transition_type):
    """Función principal para crear el video sincronizado."""
    clear_output_dir()
    status = ""
    output_video_path = None
    final_clip = None
    audio_clip = None
    image_clips = []

    if audio_file is None:
        return None, "Error: Falta el archivo de audio."
    if not image_files:
        return None, "Error: Faltan los archivos de imagen."

    audio_path = audio_file
    image_paths = [img.name for img in image_files]

    target_size = (1280, 720)
    fps = 30

    try:
        # 1. Analizar audio
        print(f"Cargando audio: {audio_path}")
        y, sr = librosa.load(audio_path, sr=None)
        audio_duration = librosa.get_duration(y=y, sr=sr)
        print(f"Duración del audio: {audio_duration:.2f} segundos")

        effective_duration = min(audio_duration, max_duration_seconds if max_duration_seconds > 0 else audio_duration)
        if effective_duration < audio_duration:
            print(f"Video limitado a {effective_duration:.2f} segundos.")
            y = y[:int(effective_duration * sr)]

        print("Detectando beats...")
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, units='frames')
        beat_times = librosa.frames_to_time(beat_frames, sr=sr)

        # Convertir tempo a escalar si es un arreglo
        if isinstance(tempo, np.ndarray):
            tempo = tempo.item() if tempo.size == 1 else tempo[0]

        print(f"Tempo estimado: {tempo:.2f} BPM")
        print(f"Número de beats detectados: {len(beat_times)}")

        if len(beat_times) == 0:
            print("No se detectaron beats, usando intervalos fijos.")
            beat_interval = 2.0
            beat_times = np.arange(0, effective_duration, beat_interval)
            if len(beat_times) == 0:
                beat_times = np.array([0.0])

        beat_times = np.append(beat_times, effective_duration)
        beat_durations = np.diff(beat_times)
        beat_durations[beat_durations <= 0] = 0.1

        # 2. Preparar imágenes
        print("Preparando imágenes...")
        resized_image_paths = []
        valid_image_paths = []
        for img_path in image_paths:
            if os.path.exists(img_path):
                resized = resize_image(img_path, target_size)
                if resized:
                    resized_image_paths.append(resized)
                    valid_image_paths.append(img_path)
            else:
                print(f"Advertencia: No se encontró el archivo de imagen {img_path}")

        if not resized_image_paths:
            raise ValueError("No se pudieron procesar las imágenes. Asegúrate de que sean archivos válidos (JPG, PNG).")

        print(f"{len(resized_image_paths)} imágenes preparadas.")

        # 3. Crear clips de imagen sincronizados
        print("Creando clips de video...")
        current_image_index = 0
        beat_index = 0
        total_beats_to_process = len(beat_durations)

        while beat_index < total_beats_to_process:
            beats_this_image = int(images_per_beat)
            duration_this_image = sum(beat_durations[beat_index:min(beat_index + beats_this_image, total_beats_to_process)])

            if duration_this_image <= 0:
                beat_index += beats_this_image
                continue

            img_path_for_clip = resized_image_paths[current_image_index % len(resized_image_paths)]
            clip = ImageClip(img_path_for_clip, duration=duration_this_image)

            if transition_type == "Fade" and image_clips:
                clip = clip.crossfadein(min(0.5, duration_this_image / 2))

            image_clips.append(clip)
            beat_index += beats_this_image
            current_image_index += 1

        if not image_clips:
            raise ValueError("No se pudieron crear clips de video. Posible problema con tiempos de beat.")

        # 4. Concatenar clips y añadir audio
        print("Concatenando clips...")
        final_clip = concatenate_videoclips(image_clips, method="compose")
        audio_clip = AudioFileClip(audio_path).set_duration(final_clip.duration)
        print("Añadiendo audio...")
        final_clip = final_clip.set_audio(audio_clip)
        final_clip = final_clip.set_duration(effective_duration)

        # 5. Escribir el video final
        output_filename = f"synced_video_{int(time.time())}.mp4"
        output_video_path = os.path.join(output_dir, output_filename)
        print(f"Guardando video final en: {output_video_path}")
        final_clip.write_videofile(
            output_video_path,
            codec='libx264',
            audio_codec='aac',
            fps=fps,
            threads=4,
            logger='bar'
        )

        status = f"¡Video sincronizado creado! ({final_clip.duration:.2f}s)."

    except ValueError as ve:
        status = f"Error de Validación: {ve}"
        print(f"Error: {ve}")
        output_video_path = None
    except Exception as e:
        status = f"Error inesperado: {e}"
        print(f"Error: {e}")
        output_video_path = None
    finally:
        # Liberar recursos
        if final_clip:
            final_clip.close()
        if audio_clip:
            audio_clip.close()
        for ic in image_clips:
            ic.close()
        # Limpiar imágenes redimensionadas
        for rp in locals().get('resized_image_paths', []):
            if os.path.exists(rp):
                os.remove(rp)

    return output_video_path, status

# Interfaz Gradio
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎬 Creador de Video Sincronizado con Música")
    gr.Markdown("Sube un archivo de audio y varias imágenes. Las imágenes cambiarán al ritmo de la música.")

    with gr.Row():
        with gr.Column(scale=1):
            audio_input = gr.Audio(label="1. Sube tu archivo de Audio (MP3, WAV)", type="filepath")
            image_input = gr.File(
                label="2. Sube tus Imágenes (JPG, PNG - selecciona varias)",
                file_count="multiple",
                file_types=["image"]
            )
            max_duration = gr.Slider(
                label="3. Duración Máxima del Video (segundos, 0=usar todo el audio)",
                minimum=0,
                maximum=600,
                value=600,
                step=10
            )
            images_per_beat_slider = gr.Slider(
                label="4. Imágenes por Beat",
                minimum=1,
                maximum=8,
                value=1,
                step=1
            )
            transition_dropdown = gr.Radio(
                label="5. Transición entre Imágenes",
                choices=["Corte Directo", "Fade"],
                value="Corte Directo"
            )
            submit_button = gr.Button("✨ Crear Video Sincronizado", variant="primary")
        with gr.Column(scale=1):
            video_output = gr.Video(label="Resultado del Video")
            status_output = gr.Textbox(label="Estado", interactive=False)

    submit_button.click(
        fn=create_synced_video,
        inputs=[audio_input, image_input, max_duration, images_per_beat_slider, transition_dropdown],
        outputs=[video_output, status_output]
    )

# Lanzar la interfaz
try:
    print("\nLanzando la interfaz de Gradio con share=True...")
    demo.launch(debug=True, share=True)
except Exception as e:
    print(f"Error al lanzar Gradio con share=True: {e}")
    print("Intentando con share=False...")
    try:
        demo.launch(debug=True, share=False)
        print("Gradio iniciado en enlace local. Usa el enlace proporcionado arriba (por ejemplo, https://*.prod.colab.dev).")
    except Exception as e2:
        print(f"Error al lanzar Gradio con share=False: {e2}")
        print("Por favor, revisa la instalación de Gradio o intenta ejecutar en una máquina local.")

Instalando librerías: gradio, moviepy, librosa, Pillow...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
Instalación completada.


  if event.key is 'enter':



Librerías importadas.

Lanzando la interfaz de Gradio con share=True...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b084fc5521aaba9245.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Cargando audio: /tmp/gradio/03beaf584299f132d0612c7b053191cb1345a1d3586093e00134c80c02f2857a/Afrojack  Steve Aoki ft Miss Palmer - No Beef Official Music Video.mp3
Duración del audio: 333.34 segundos
Detectando beats...
Tempo estimado: 129.20 BPM
Número de beats detectados: 691
Preparando imágenes...
9 imágenes preparadas.
Creando clips de video...
Concatenando clips...
Añadiendo audio...
Guardando video final en: synced_video_outputs/synced_video_1745938502.mp4
Moviepy - Building video synced_video_outputs/synced_video_1745938502.mp4.
MoviePy - Writing audio in synced_video_1745938502TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video synced_video_outputs/synced_video_1745938502.mp4



Moviepy - Done !
Moviepy - video ready synced_video_outputs/synced_video_1745938502.mp4
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b084fc5521aaba9245.gradio.live
